In [1]:
 ! git clone https://github.com/WongKinYiu/yolov7

fatal: destination path 'yolov7' already exists and is not an empty directory.


In [2]:
cd yolov7

C:\Users\HP\tut\Indoor object detection\yolov7


In [3]:
#pip install -r requirements.txt


In [ ]:
import torch
import yaml
from yolov7.models.yolo import Model
from yolov7.utils.datasets import create_dataloader
from yolov7.utils.general import check_img_size, increment_path
from yolov7.utils.loss import ComputeLoss
from yolov7.utils.torch_utils import select_device, train_one_epoch


In [ ]:
# Set paths and device
data_yaml_path = 'C:/Users/HP/Desktop/data/data.yaml'
weights_path = 'C:/Users/HP/Desktop/data/data/yolov7.pt'  # Pre-trained weights
device = select_device('0')  # '0' for GPU, '' for CPU

In [ ]:
# Load dataset information
with open(data_yaml_path, 'r') as file:
    data = yaml.safe_load(file)

In [ ]:
# Model configuration
nc = data['nc']  # Number of classes
img_size = 640   # Image size (can be adjusted)
batch_size = 16  # Batch size (adjust based on your hardware)
epochs = 50      # Number of epochs
hyp = {
    'lr0': 0.01,  # Initial learning rate
    'momentum': 0.937,
    'weight_decay': 0.0005,
    # Add other hyperparameters as needed
}

In [ ]:
# Load model
model = Model(cfg='C:/Users/HP/Desktop/data/yolov7.yaml', nc=nc).to(device)
model.load_state_dict(torch.load(weights_path, map_location=device)['model'])
model.half()  # To use mixed precision if supported

In [ ]:
# Create dataloaders
train_loader, dataset = create_dataloader(data['train'], img_size, batch_size, 32, hyp, augment=True, cache=False, rect=False)
val_loader = create_dataloader(data['val'], img_size, batch_size, 32, hyp, augment=False, cache=False, rect=False)[0]

In [ ]:
# Optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=hyp['lr0'], momentum=hyp['momentum'], weight_decay=hyp['weight_decay'])
criterion = ComputeLoss(model)

In [ ]:
# Train the model
for epoch in range(epochs):
    model.train()
    for imgs, targets, paths, _ in train_loader:
        imgs = imgs.to(device).half()
        targets = targets.to(device)
        preds = model(imgs)
        loss, loss_items = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Optionally, validate the model on the validation set
    model.eval()
    # Validation code here...

    # Save model checkpoint
    torch.save(model.state_dict(), f'model_epoch_{epoch}.pt')


In [ ]:
# Save final model
torch.save(model.state_dict(), 'final_model.pt')